# Univariate prediction and persistence models

This notebook consolidates the univariate onset-prediction workflows with the cleaned persistence/remission cohort. Both sections reuse the shared helpers so the preprocessing, cohort definitions, and logistic regression pipelines stay synchronized with the multivariate analyses.

In [ ]:
import urllib.request
from pathlib import Path

BASE_URL = "https://raw.githubusercontent.com/FelixZhan/AtyAN/main/"
HELPER_FILES = [
    "analysis_utils.py",
    "requirements.txt",
    "BP1234-ONSET.csv",
]

for filename in HELPER_FILES:
    dest = Path(filename)
    if dest.exists():
        print(f'{filename} already present, skipping download.')
        continue
    print(f'Downloading {filename}...')
    urllib.request.urlretrieve(f"{BASE_URL}{filename}", dest)
print('Helper files are ready.')


In [ ]:
!pip install -q -r requirements.txt


## Imports and shared setup

In [ ]:
from analysis_utils import (
    load_base_dataset,
    engineer_baseline_features,
    prepare_univariate_prediction_dataset,
    prepare_persistence_dataset,
    run_univariate_logistic_regressions,
    evaluate_model_zoo,
)
from IPython.display import display


In [ ]:
model_subset = []  # e.g., ['tabpfn_random_forest']


In [ ]:
raw_df = load_base_dataset()
feature_df, feature_sets = engineer_baseline_features(raw_df)
print(f'Dataset shape: {raw_df.shape}')
print(f'Feature matrix shape: {feature_df[feature_sets["all_features"]].shape}')


## Univariate prediction of future atypical AN onset

Participants with full AN diagnoses or baseline atypical AN onset are removed to mirror the original risk-prediction experiment. The target labels any mBMI-defined atypical AN onset across waves 1–6.

In [ ]:
prediction_df = prepare_univariate_prediction_dataset(
    feature_df, feature_sets['all_features']
)
outcome_counts = prediction_df['aan_onset_anywave'].value_counts().to_dict()
print('Univariate prediction cohort size:', len(prediction_df))
print('Outcome counts:', outcome_counts)
prediction_results = run_univariate_logistic_regressions(
    prediction_df, feature_sets['all_features'], target_col='aan_onset_anywave'
)
display(prediction_results)


## Univariate persistence vs. remission analyses

The persistence dataset retains participants with baseline or mBMI-defined onset who have complete wave-1–6 onset data and labels cases that revisit onset after at least one remission wave.

In [ ]:
persistence_df = prepare_persistence_dataset(
    feature_df, feature_sets['all_features']
)
print('Persistence cohort size:', len(persistence_df))
print(persistence_df['aan_persistence'].value_counts().rename('count'))
persistence_results = run_univariate_logistic_regressions(
    persistence_df, feature_sets['all_features']
)
display(persistence_results)


### Onset logistic summaries


In [ ]:
onset_logistic = run_univariate_logistic_regressions(
    prediction_df,
    feature_sets['all_features'],
    target_col='aan_onset_anywave',
)
display(onset_logistic.head(20))


### Onset model zoo


In [ ]:
onset_model_metrics, onset_split_tables, _, onset_errors = evaluate_model_zoo(
    prediction_df,
    feature_sets['all_features'],
    target_col='aan_onset_anywave',
    model_names=model_subset or None,
)
display(onset_model_metrics)
if not onset_model_metrics.empty and 'overfit_flag' in onset_model_metrics.columns:
    flagged = onset_model_metrics[onset_model_metrics['overfit_flag']]
    if not flagged.empty:
        for name in flagged['model']:
            display(onset_split_tables[name])
for name, err in onset_errors.items():
    print(f'{name}: {err}')


### Persistence logistic summaries


In [ ]:
persistence_logistic = run_univariate_logistic_regressions(
    persistence_df,
    feature_sets['all_features'],
    target_col='aan_persistence',
)
display(persistence_logistic.head(20))


### Persistence model zoo


In [ ]:
persistence_model_metrics, persistence_split_tables, persistence_feature_tables, persistence_errors = evaluate_model_zoo(
    persistence_df,
    feature_sets['all_features'],
    target_col='aan_persistence',
    model_names=model_subset or None,
)
display(persistence_model_metrics)
if not persistence_model_metrics.empty and 'overfit_flag' in persistence_model_metrics.columns:
    flagged = persistence_model_metrics[persistence_model_metrics['overfit_flag']]
    if not flagged.empty:
        for name in flagged['model']:
            display(persistence_split_tables[name])
for label, table in persistence_feature_tables.items():
    display(table.head(20))
for name, err in persistence_errors.items():
    print(f'{name}: {err}')
